# Packages 

In [42]:
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import glob
import pathlib
import unicodedata
import tarfile

from pandarallel import pandarallel
pandarallel.initialize()
from unidecode import unidecode

def get_csv_size(csv_name):
    get_size = os.path.getsize(os.getcwd() + '/'+csv_name)
    mb_size = get_size/(1024 * 1024)
    mb_size = round(mb_size,1)
    return mb_size

def get_attribute(filename):
    if filename.endswith('csv'):
        df1 = pd.read_csv(filename,header=[0])
    else:
        df1=pd.read_json(filename,orient="records", lines=True)
    return list(df1.columns) 

def split_large_file(filename, source, output,size=None):
    df1=pd.read_csv(filename,header=[0])
    if size ==None:
        size = get_csv_size(filename)
    num_chunks = size//20
    if num_chunks == 0:
        num_chunks = 1 
    df_all = np.array_split(df1, num_chunks)

    for idx, file in enumerate(df_all):
        file.to_csv('%s\\%s_%03d.csv'%(output, source, idx), index=False)   
        
def show_all_files(folder):
    df = pd.DataFrame(glob.glob('%s/*'%folder), columns = ['path'])
    df['root'] = df.path.apply(lambda x: x.split('\\')[0])
    df['source'] = df.path.apply(lambda x: x.replace('%s\\'%folder,'').split('_')[0])
    df['filename'] = df.path.apply(lambda x: x.replace('%s\\'%folder,'').split('/')[-1])
    df = df.sort_values('source').reset_index(drop=True)
    df['size'] = df['path'].apply(get_csv_size)
    return df

def unicodetoascii(text):
    TEXT = (text.
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').#
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")"))
    return TEXT

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


# Check csv

In [3]:
pandarallel.initialize(nb_workers=1, progress_bar=False) 

df = pd.DataFrame(glob.glob('6/*/*.csv'), columns = ['path'])

df['source'] = df.path.apply(lambda x: x.split('\\')[1].replace('.csv', ''))
df = df.sort_values('source').reset_index(drop=True)
df['attributes']= df.path.apply(get_attribute)
df['size']= df.path.apply(get_csv_size)
df

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


,path,source,attributes,size
0,6\diaadia\diaadia.csv,diaadia,"[page, rows, link, text]",48.9
1,6\diaadia\diaadia_filtered.csv,diaadia,"[Unnamed: 0, page, rows, link, text]",18.6
2,6\elpanam\elpanam.csv,elpanam,"[page, rows, link, text]",2.2
3,6\elpanam\elpanam_filtered.csv,elpanam,"[Unnamed: 0, page, rows, link, text]",0.5
4,6\lacritica\lacritica.csv,lacritica,"[page, rows, link, text]",10.1
5,6\lacritica\lacritica_filtered.csv,lacritica,"[Unnamed: 0, page, rows, link, text]",3.9
6,6\laprensa\laprensa.csv,laprensa,"[page, rows, link, text]",4.3
7,6\laprensa\laprensa_filtered.csv,laprensa,"[Unnamed: 0, page, rows, link, text]",0.0


In [4]:
with open('wiki_relevant_spanish.txt', 'r', encoding='utf8') as file:    
    relevant_rules = file.read().split('\n')
relevant_rules = '\\b' + '|\\b'.join(relevant_rules)

if relevant_rules.endswith('|\\b'):
    relevant_rules =relevant_rules[:-3]   

#relevant_rules = re.compile(relevant_rules, re.IGNORECASE)

print('relevant_rules:')
print(relevant_rules)

with open('irrelevant_keywords_spanish.txt', 'r', encoding='utf8') as file:    
    irelevant_rules = file.read().split('\n')
irelevant_rules = '\\b' + '|\\b'.join(irelevant_rules)

if irelevant_rules.endswith('|\\b'):
    irelevant_rules =irelevant_rules[:-3]   

#irelevant_rules = re.compile(irelevant_rules, re.IGNORECASE)

print('\nirelevant_rules:')
print(irelevant_rules)


find_relevant = lambda x: Counter(relevant_rules.findall(str(x).lower()))
find_exclude = lambda x: Counter(irelevant_rules.findall(str(x).lower()))

relevant_rules:
\brelaciones \w+-\w+|\bsecuestrar|\babuso|\bactivista|\bactos\b|\badministración|\bcontra las mujeres|\bagencias|\ballegati|\bembajador|\banarquía|\barmado|\bejército|\bdetención|\bartillería|\basesino|\basalto|\basamblea|\basilo|\bataque|\battrocit|\bautoridades|\bboleta electoral|\bbatallas...|\bderramamiento de sangre|\bBLOTTER|\bcuerpos|\bbomba|\bfrontera|\bboicot|\barmarios|\bcaso\b|\bcasualt|\bbajas|\bcensores|\bcivil\b|\bcoaccionar|\bcolonial|\bcombate|\bcomité|\bmancomunidad|\bcomunismo|\bcomunista|\bconflicto|\bcongreso|\bconscripto|\bconservador|\bconstituenc|\bconstitución|\bconvicto|\bcorps\b|\bcorrupto|\bconsejo|\bconcejal|\bContrainsurgencia|\bContra el terrorismo|\bPaíses en|\bgolpes de estado|\btribunales|\bdelitos?|\bcriminal|\brestringir|\bciberataque|\bcibercrimen|\bdaño|\bmuerte|\bmuerte|\bdefensa|\bdefensa|\bdelegado|\bdemocrático|\bdemostración|\bdepartamento|\brepresentación|\bdesplegar|\bdeportar|\bdiputado|\bdestruir|\bdetener|\bdetención|\bdict

In [5]:
find_relevant = lambda x: Counter(relevant_rules.findall(x.lower()))
find_exclude = lambda x: Counter(irelevant_rules.findall(x.lower()))
for path in df.path:
    df1 = pd.read_csv(path, header=[0])
    df1 = df1[~df1.text.isnull()]
    df1 = df1[~(df1.text.str.count(irelevant_rules) > 1)]
    print(df1.info())
    df1.to_csv(path[:-4] + '_filtered.csv')

KeyboardInterrupt: 

In [6]:
df['path'] = df['path'].apply(lambda x: x[:-4] + '_filtered.csv')

In [9]:
df = df[~df.path.str.contains('_filtered_filtered')]

# Split

In [10]:
pandarallel.initialize(nb_workers=8, progress_bar=False)       
_ = df.apply(lambda x: split_large_file(x['path'],x['source'],'split'), axis=1) 

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [11]:
df = show_all_files('split')
df

,path,root,source,filename,size
0,split\diaadia_000.csv,split,diaadia,diaadia_000.csv,18.6
1,split\elpanam_000.csv,split,elpanam,elpanam_000.csv,0.5
2,split\lacritica_000.csv,split,lacritica,lacritica_000.csv,3.9
3,split\laprensa_000.csv,split,laprensa,laprensa_000.csv,0.0


In [9]:
df['size'].describe()

count     4.000000
mean      5.750000
std       8.740137
min       0.000000
25%       0.375000
50%       2.200000
75%       7.575000
max      18.600000
Name: size, dtype: float64

# step 1  clean

In [12]:
df = show_all_files('split')
# df['attribute'] = df.path.parallel_apply(get_attribute)
df

,path,root,source,filename,size
0,split\diaadia_000.csv,split,diaadia,diaadia_000.csv,18.6
1,split\elpanam_000.csv,split,elpanam,elpanam_000.csv,0.5
2,split\lacritica_000.csv,split,lacritica,lacritica_000.csv,3.9
3,split\laprensa_000.csv,split,laprensa,laprensa_000.csv,0.0


In [11]:
df.source.unique()

array(['diaadia', 'elpanam', 'lacritica', 'laprensa'], dtype=object)

In [13]:
print('total size:', df['size'].sum())
df.groupby(by=["source"], dropna=False).sum()

total size: 23.0


,size
source,
diaadia,18.6
elpanam,0.5
lacritica,3.9
laprensa,0.0


In [14]:
def basic_process(filename, output_folder):
    
    sizes = []
    
    df1 = pd.read_csv(filename, header=[0])
    sizes.append(df1.shape[0])
    
    if 'url' not in df1.columns:
        df1['url'] ='\n'
    if 'title' not in df1.columns:
        df1['title']='\n'
    
#     df1 = df1[~df1.url.isnull()]
    
    df1 = df1[~df1.text.isnull()]
    df1 = df1[~df1.text.duplicated()]
#     df1 = df1[~df1.url.duplicated()]
    
    df1.loc[df1.title.isnull(),'title']='\n'
    df1.loc[df1.url.isnull(),'url']='\n'
    if 'abstract' in df1:
        df1.loc[df1.abstract.isnull(),'abstract']='\n'
        
    # ------------------- Start cleaning  --------------------------#
    
    # Convert coding
    df1.text = df1.text.apply(lambda x: unicodetoascii(x))
    df1.text = df1.text.apply(lambda x: unicodedata.normalize("NFKD", x))
    
    # email
    df1.text = df1.text.apply(lambda x: re.sub("\S+@\S+(?:\.\S+)+",'',x))
    
    # telphone
    df1.text = df1.text.apply(lambda x: re.sub('\(\+( |-|\d)+\)( |-|\d)+',' ',x))
    df1.text = df1.text.apply(lambda x: re.sub('\+( |-|\d)+',' ',x))
    
    # noise
    df1.text =\
    df1.text.apply(lambda x: re.sub('\n(ad|advertisement|tweet):?\n', "", x, flags=re.IGNORECASE))
    
    # urls
    df1.text = df1.text.apply(lambda x: re.sub(r"http\S+", "", x))
    
    # delete too many \n
    df1.text = df1.text.apply(lambda x: re.sub('\n\n+', "\n\n", x, flags=re.IGNORECASE))
    
    # head and tails
    df1.text = df1.text.apply(lambda x: re.sub("^\s+|\s+$", "", x, flags=re.UNICODE)) 
    
    
    df1 = df1[df1.text.str.len()>100]
    
    sizes.append(df1.shape[0])
    
    # ------------------- Ending cleaning  --------------------------#
    
    filename = filename.split('\\')[1]
    new_filename = output_folder + '/'+ filename

    print('%s:\t%s'%(filename, sizes))
    df1.to_csv(new_filename, index= False)
    return

In [15]:
pandarallel.initialize(nb_workers=8, progress_bar=False) 
_ = df['path'].apply(basic_process, output_folder='step1')

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/
diaadia_000.csv:	[10181, 10049]
elpanam_000.csv:	[180, 180]
lacritica_000.csv:	[1341, 1329]
laprensa_000.csv:	[0, 0]


In [15]:
df = show_all_files('step1')
df

,path,root,source,filename,size
0,step1\diaadia_000.csv,step1,diaadia,diaadia_000.csv,18.7
1,step1\elpanam_000.csv,step1,elpanam,elpanam_000.csv,0.5
2,step1\lacritica_000.csv,step1,lacritica,lacritica_000.csv,3.9
3,step1\laprensa_000.csv,step1,laprensa,laprensa_000.csv,0.0


In [16]:
print('total size:', df['size'].sum())
df.groupby(by=["source"], dropna=False).sum()

total size: 23.099999999999998


,size
source,
diaadia,18.7
elpanam,0.5
lacritica,3.9
laprensa,0.0


# save to json

In [50]:
# folder = '2.Organization'
# df = pd.DataFrame(columns=['path','source', 'filename'])
# df.path = [str(x) for x in pathlib.Path('%s/*'%folder).glob('**/*')]
df = pd.DataFrame(glob.glob('*\\*\\*'), columns = ['path'])
df = df[df.path.str.contains('_filtered')]
# df['root'] = df.path.apply(lambda x: x.split('/')[0])

df['source'] = df.path.apply(lambda x: x.split('\\')[1])
df['filename'] = df.path.apply(lambda x: x.split('\\')[-1])

# df = df.sort_values('source').reset_index(drop=True)
# df['size'] = df['path'].parallel_apply(get_csv_size)
# df['filename'] = df.path.apply(lambda x: x.replace('%s/'%folder,''))

df['json_file'] = df['filename'].apply(lambda x: x.replace('.csv', '.json'))

df['json_file'] = df.source +'\\'+ df.json_file
df['tar_file'] = 'tar\\'+ df.json_file+'.tar.gz'
df.json_file = 'json\\'+ df.json_file
df

,path,source,filename,json_file,tar_file
1,6\diaadia\diaadia_filtered.csv,diaadia,diaadia_filtered.csv,json\diaadia\diaadia_filtered.json,tar\diaadia\diaadia_filtered.json.tar.gz
3,6\elpanam\elpanam_filtered.csv,elpanam,elpanam_filtered.csv,json\elpanam\elpanam_filtered.json,tar\elpanam\elpanam_filtered.json.tar.gz
5,6\lacritica\lacritica_filtered.csv,lacritica,lacritica_filtered.csv,json\lacritica\lacritica_filtered.json,tar\lacritica\lacritica_filtered.json.tar.gz
7,6\laprensa\laprensa_filtered.csv,laprensa,laprensa_filtered.csv,json\laprensa\laprensa_filtered.json,tar\laprensa\laprensa_filtered.json.tar.gz
8,json\diaadia\diaadia_filtered.json,diaadia,diaadia_filtered.json,json\diaadia\diaadia_filtered.json,tar\diaadia\diaadia_filtered.json.tar.gz
9,json\elpanam\elpanam_filtered.json,elpanam,elpanam_filtered.json,json\elpanam\elpanam_filtered.json,tar\elpanam\elpanam_filtered.json.tar.gz
10,json\lacritica\lacritica_filtered.json,lacritica,lacritica_filtered.json,json\lacritica\lacritica_filtered.json,tar\lacritica\lacritica_filtered.json.tar.gz
11,json\laprensa\laprensa_filtered.json,laprensa,laprensa_filtered.json,json\laprensa\laprensa_filtered.json,tar\laprensa\laprensa_filtered.json.tar.gz
12,tar\diaadia\diaadia_filtered.json.tar.gz,diaadia,diaadia_filtered.json.tar.gz,json\diaadia\diaadia_filtered.json.tar.gz,tar\diaadia\diaadia_filtered.json.tar.gz.tar.gz
13,tar\elpanam\elpanam_filtered.json.tar.gz,elpanam,elpanam_filtered.json.tar.gz,json\elpanam\elpanam_filtered.json.tar.gz,tar\elpanam\elpanam_filtered.json.tar.gz.tar.gz


In [25]:
cwd = os.getcwd()
cwd

'C:\\Users\\vikas\\OneDrive\\Documents\\GitHub\\ConfliBERT\\pretrain-corpora\\Crawlers and Process\\Preprocess'

In [27]:
os.mkdir(cwd + '\\json\\')
os.mkdir(cwd + '\\tar\\')

for i in df.source.unique():
    os.mkdir(cwd + '\\json\\'+i)
    os.mkdir(cwd + '\\tar\\'+i)  

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\vikas\\OneDrive\\Documents\\GitHub\\ConfliBERT\\pretrain-corpora\\Crawlers and Process\\Preprocess\\json\\'

In [31]:
print(df)
df1 = pd.read_csv(df.path[1], header=[0])
df1

                                 path     source                filename  \
1      6\diaadia\diaadia_filtered.csv    diaadia    diaadia_filtered.csv   
3      6\elpanam\elpanam_filtered.csv    elpanam    elpanam_filtered.csv   
5  6\lacritica\lacritica_filtered.csv  lacritica  lacritica_filtered.csv   
7    6\laprensa\laprensa_filtered.csv   laprensa   laprensa_filtered.csv   

                                json_file  \
1      json/diaadia/diaadia_filtered.json   
3      json/elpanam/elpanam_filtered.json   
5  json/lacritica/lacritica_filtered.json   
7    json/laprensa/laprensa_filtered.json   

                                       tar_file  
1      tar/diaadia/diaadia_filtered.json.tar.gz  
3      tar/elpanam/elpanam_filtered.json.tar.gz  
5  tar/lacritica/lacritica_filtered.json.tar.gz  
7    tar/laprensa/laprensa_filtered.json.tar.gz  


,Unnamed: 0,page,rows,link,text
0,5,https://www.diaadia.com.pa/mundo?page=0,"<div class=""col-12 col-md-6"">\n<div class="" mb-3 card_secciones "">\n<div class=""card"">\n<a href=""/mundo/la-oms-evalua-distribucion-de-vacunas-para-contener-brote-de-viruela-del-mono-751451"">\n<img src="" /sites/default/files/styles/secciones/public/2022-06/monos.jpg?itok=dHGK0udz"" width=""100%""/>\n</a>\n<div class=""card-body-fixed"">\n<div class=""card-title custom"">\n<div class=""bg-blanco"">\n<span class=""categoria-bg"">Mundo </span>\n</div>\n<a class=""hovernaranja"" href=""/mundo/la-oms-evalua-distribucion-de-vacunas-para-contener-brote-de-viruela-del-mono-751451"">\n<h5 class=""title-fixed der"">L...",https://www.diaadia.com.pa/mundo/la-oms-evalua-distribucion-de-vacunas-para-contener-brote-de-viruela-del-mono-751451,"La Organización Mundial de la Salud (OMS) dijo este miércoles que está haciendo una evaluación de la cantidad y eficacia de vacunas contra la viruela que están disponibles para distribuirlas en cantidades relativamente pequeñas a los países donde se han detectado casos de la viruela del mono.\n\n\n\n""Hay antivirales y vacunas aprobadas para la viruela del mono pero su disponibilidad es limitada y estamos coordinado la distribución de las reservas en función de la equidad y las necesidades de salud pública"", explicó el director general de la OMS, Tedros Adhanom Ghebreyesus.\n\n\n\nCon ese f..."
1,6,https://www.diaadia.com.pa/mundo?page=0,"<div class=""col-12 col-md-6"">\n<div class="" mb-3 card_secciones "">\n<div class=""card"">\n<a href=""/mundo/moderna-dice-que-su-vacuna-revisada-crea-mayor-respuesta-inmune-ante-omicron-751447"">\n<img src="" /sites/default/files/styles/secciones/public/2022-06/moder.jpg?itok=kdpWMggP"" width=""100%""/>\n</a>\n<div class=""card-body-fixed"">\n<div class=""card-title custom"">\n<div class=""bg-blanco"">\n<span class=""categoria-bg"">Mundo </span>\n</div>\n<a class=""hovernaranja"" href=""/mundo/moderna-dice-que-su-vacuna-revisada-crea-mayor-respuesta-inmune-ante-omicron-751447"">\n<h5 class=""title-fixed der"">Mod...",https://www.diaadia.com.pa/mundo/moderna-dice-que-su-vacuna-revisada-crea-mayor-respuesta-inmune-ante-omicron-751447,"La farmacéutica Moderna anunció este miércoles que su vacuna revisada contra el covid-19, que todavía está en periodo de prueba, produce una mayor respuesta inmune contra la variante ómicron del coronavirus y que espera que pueda estar disponible como dosis de refuerzo para este otoño.\n\n\n\nSegún los ensayos clínicos de la compañía, la nueva dosis de refuerzo de 50 microgramos ""cumplió con todos los criterios de valoración preespecificados, incluida una respuesta de anticuerpos neutralizantes superior frente a la variante ómicron un mes después de la administración, en comparación con la..."
2,12,https://www.diaadia.com.pa/mundo?page=1,"<div class=""col-12 col-md-6"">\n<div class="" mb-3 card_secciones "">\n<div class=""card"">\n<a href=""/mundo/episcopados-de-nicaragua-costa-rica-y-panama-respaldan-obispo-recluido-750614"">\n<img src="" /sites/default/files/styles/secciones/public/2022-05/nica%20.jpg?itok=f2IOnFVe"" width=""100%""/>\n</a>\n<div class=""card-body-fixed"">\n<div class=""card-title custom"">\n<div class=""bg-blanco"">\n<span class=""categoria-bg"">Mundo </span>\n</div>\n<a class=""hovernaranja"" href=""/mundo/episcopados-de-nicaragua-costa-rica-y-panama-respaldan-obispo-recluido-750614"">\n<h5 class=""title-fixed der"">Episcopados d...",https://www.diaadia.com.pa/mundo/episcopados-de-nicaragua-costa-rica-y-panama-respaldan-obispo-recluido-750614,"Los Episcopados de Nicaragua, Costa Rica y Panamá expresaron públicamente su solidaridad y respaldo este domingo al obispo nicaragüense Rolando Álvarez, crítico del Gobierno de Daniel Ortega, quien se encuentra “recluido” en una parroquia de Managua, y en ayuno indefinido, hasta que la Policía Nacional se comprometa a suspender la “persecución” en su contra.\n\n\n\n“Expresamos nuestra solidarid

In [51]:
def convert_json_tar(filename, json_file, tar_file):
    df1 = pd.read_csv(filename, header=[0])      
    df1.to_json(json_file, orient="records", lines=True)
    
    with tarfile.open(tar_file, "w:gz") as tar:
        tar.add(json_file, arcname=os.path.basename(json_file))
df.apply(lambda x: convert_json_tar(x['path'], x['json_file'], x['tar_file']), axis=1)

ParserError: Error tokenizing data. C error: Expected 35 fields in line 3, saw 43


In [52]:
pandarallel.initialize() 

df.source = df.path.apply(lambda x: x.split('\\')[1]).values
df['size'] = df['path'].apply(get_csv_size)
df

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


,path,source,filename,json_file,tar_file,size
1,6\diaadia\diaadia_filtered.csv,diaadia,diaadia_filtered.csv,json\diaadia\diaadia_filtered.json,tar\diaadia\diaadia_filtered.json.tar.gz,18.6
3,6\elpanam\elpanam_filtered.csv,elpanam,elpanam_filtered.csv,json\elpanam\elpanam_filtered.json,tar\elpanam\elpanam_filtered.json.tar.gz,0.5
5,6\lacritica\lacritica_filtered.csv,lacritica,lacritica_filtered.csv,json\lacritica\lacritica_filtered.json,tar\lacritica\lacritica_filtered.json.tar.gz,3.9
7,6\laprensa\laprensa_filtered.csv,laprensa,laprensa_filtered.csv,json\laprensa\laprensa_filtered.json,tar\laprensa\laprensa_filtered.json.tar.gz,0.0
8,json\diaadia\diaadia_filtered.json,diaadia,diaadia_filtered.json,json\diaadia\diaadia_filtered.json,tar\diaadia\diaadia_filtered.json.tar.gz,20.5
9,json\elpanam\elpanam_filtered.json,elpanam,elpanam_filtered.json,json\elpanam\elpanam_filtered.json,tar\elpanam\elpanam_filtered.json.tar.gz,0.6
10,json\lacritica\lacritica_filtered.json,lacritica,lacritica_filtered.json,json\lacritica\lacritica_filtered.json,tar\lacritica\lacritica_filtered.json.tar.gz,4.2
11,json\laprensa\laprensa_filtered.json,laprensa,laprensa_filtered.json,json\laprensa\laprensa_filtered.json,tar\laprensa\laprensa_filtered.json.tar.gz,0.0
12,tar\diaadia\diaadia_filtered.json.tar.gz,diaadia,diaadia_filtered.json.tar.gz,json\diaadia\diaadia_filtered.json.tar.gz,tar\diaadia\diaadia_filtered.json.tar.gz.tar.gz,4.9
13,tar\elpanam\elpanam_filtered.json.tar.gz,elpanam,elpanam_filtered.json.tar.gz,json\elpanam\elpanam_filtered.json.tar.gz,tar\elpanam\elpanam_filtered.json.tar.gz.tar.gz,0.2


In [53]:
print(df['size'].sum())
df.groupby(by=["source"], dropna=False)['size'].sum().round(1)

54.199999999999996


source
diaadia      44.0
elpanam       1.3
lacritica     8.9
laprensa      0.0
Name: size, dtype: float64